Bu projemizde CRISP - DM (Cross Industry Standard Process for Data Mining) metodolojisi temel alınacaktır.

### 1. Business Understanding (İş Problemini Anlamak)

Bir kişinin önceden belirlenmiş olan randevuya gelip gelmeyeceğinin tahmin edilmesi istenmektedir.
Bu problem için, gözetimli öğrenme yöntemlerinden sınıflandırma problemleri için kullanılan LogisticRegression,
DecisionTree, RandomForest, SVM (SupportVectorMachine) vb. modeller kullanılabilir. 

Gözetimsiz öğrenme yöntemlerinden yararlanmak istenirse de,  randevusuna gidecekler ve gitmeyecekler olarak hastaları kümelere ayırma veya segmente etme işlemi gerçekleştirilebilir. Bunun için de makine öğrenmesi modellerinden olan K-Ortalama Kümeleme (K-Mean Clustering) veya Yoğunluk Tabanlı Kümeleme (DBSCAN(density-based clustering non-parametric algorithm)) kullanılabilir.

### 2. Data Understanding(Veri Setini Anlamak)

### 3. Data Preparation(Veriyi Hazırlama)

### 4. Modeling (Modelleme)

### 5. Evaluation(Modeli Değerlendirme)

### 6. Deployment (Dağıtım)



In [1]:
# import libraries
import pandas as pd # data manipulation, load dataset etc.
import numpy as np # linear algebra
import seaborn as sns # data visualization
import matplotlib.pyplot as plt # data visualization

# Preprocessing (Veri Ön İşleme)
from sklearn.preprocessing import StandardScaler # preprocessing

# Modelling (Modelleme için)
from sklearn.linear_model import LogisticRegression # Logistic Regression
from sklearn.svm import SVC # Support Vector Machine Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier # Decision Tree Classifier

# train-test split
from sklearn.model_selection import train_test_split

# Model Evaluation (Model Değerlendirme)
# confussion matrix, auc, f1 score etc
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay


In [8]:
# 2. Data Understanding

"""
import kagglehub
# Download latest version
path = kagglehub.dataset_download("joniarroba/noshowappointments")
print("Path to dataset files:", path)    
"""
dataset_path = r"D:\10MillionAI\Project\medical_appointment.csv"

medical_appointment = pd.read_csv(dataset_path)

if medical_appointment is None:
    raise FileNotFoundError("File path is not found.")

df = pd.DataFrame(medical_appointment)
# Show first 10 data
df.head(10)

def df_overview(dataframe):
    print("************************** Shape **************************")
    print(dataframe.shape)
    print(f"Total Observation: {dataframe.shape[0]}")
    print(f"Total Number of Features: {dataframe.shape[1]}")
    print("************************** Info **************************")
    print(dataframe.info)
    print("************************** Columns **************************")
    print(dataframe.columns)
    print("************************** Describe **************************")
    print(dataframe.describe())
    

In [13]:
# call the df_overview (Veri setimize genel bakış atalım)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)

df_overview(dataframe=df)

************************** Shape **************************
(110527, 14)
Total Observation: 110527
Total Number of Features: 14
************************** Info **************************
<bound method DataFrame.info of            PatientId  AppointmentID Gender          ScheduledDay  \
0       2.987250e+13        5642903      F  2016-04-29T18:38:08Z   
1       5.589978e+14        5642503      M  2016-04-29T16:08:27Z   
2       4.262962e+12        5642549      F  2016-04-29T16:19:04Z   
3       8.679512e+11        5642828      F  2016-04-29T17:29:31Z   
4       8.841186e+12        5642494      F  2016-04-29T16:07:23Z   
...              ...            ...    ...                   ...   
110522  2.572134e+12        5651768      F  2016-05-03T09:15:35Z   
110523  3.596266e+12        5650093      F  2016-05-03T07:27:33Z   
110524  1.557663e+13        5630692      F  2016-04-27T16:03:52Z   
110525  9.213493e+13        5630323      F  2016-04-27T15:09:23Z   
110526  3.775115e+14        56294

In [23]:
# col list comp 
[col for col in df.columns]
""" 
for col in df.columns:
    print(col)
"""
# [df[col].dtype for col in df.column
def show_col_dtype(dataframe):
    for col in dataframe.columns:
        print(col,"'s dtype is: ", dataframe[col].dtype)
    
# call the function
show_col_dtype(dataframe = df)

PatientId 's dtype is:  float64
AppointmentID 's dtype is:  int64
Gender 's dtype is:  object
ScheduledDay 's dtype is:  object
AppointmentDay 's dtype is:  object
Age 's dtype is:  int64
Neighbourhood 's dtype is:  object
Scholarship 's dtype is:  int64
Hipertension 's dtype is:  int64
Diabetes 's dtype is:  int64
Alcoholism 's dtype is:  int64
Handcap 's dtype is:  int64
SMS_received 's dtype is:  int64
No-show 's dtype is:  object


In [28]:
cat_cols = [col for col in df.columns if df[col].dtype == "O"]
num_cols = [col for col in df.columns if df[col].dtype in ['int64', 'float64']]

print("Categorical Columns: ", cat_cols)
print("*"*100)
print("Numerical Columns: ", num_cols)

Categorical Columns:  ['Gender', 'ScheduledDay', 'AppointmentDay', 'Neighbourhood', 'No-show']
****************************************************************************************************
Numerical Columns:  ['PatientId', 'AppointmentID', 'Age', 'Scholarship', 'Hipertension', 'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received']
